In [145]:
#  1. Importaciones y rutas base
import os
import numpy as np
import pandas as pd
from pathlib import Path
from pvlib.location import Location
import bifacial_radiance
import matplotlib.pyplot as plt
import bifacial_radiance.load as load
from bifacial_radiance import *
import pvlib

In [146]:
# === Configuración ===
testfolder = Path().resolve()
lat, lon = -21.01, -68.38
albedo=0.7
strategy = '80pct'  # puede ser: 'maximize_irradiance', 'tracking_classic', 'albedo_sensitive'
tz = 'America/Santiago'
print(f" Carpeta de trabajo: {testfolder}")

 Carpeta de trabajo: C:\Users\Alvaro Henriquez\OneDrive - fraunhofer.cl\Escritorio\FCR\Bifacial Radiance Simulations\BifacialSimulationsSWC2025\Post_SWC\Pruebas_SAB


In [147]:
#  2. Cargar archivo TMY personalizado desde CSV
tmy_csv_path = testfolder / "TMY_-21.01170_-68.38992.csv"
met_df = pd.read_csv(tmy_csv_path)
met_df.rename(columns={'time': 'datetime'}, inplace=True)
met_df['datetime'] = pd.to_datetime(met_df['datetime'], utc=True)
met_df.set_index('datetime', inplace=True)
met_df = met_df.tz_convert(tz)
print(" Archivo TMY personalizado cargado y ajustado de zona horaria.")

 Archivo TMY personalizado cargado y ajustado de zona horaria.


In [148]:
met_df

,temp_air,ghi,dni,dhi,lwr_u,wind_speed,rh,sp,azimuth,zenith,apparent_zenith,surface_azimuth,surface_tilt,PV_ac_W
datetime,,,,,,,,,,,,,,
2022-01-01 00:00:00-03:00,5.39,0.00,0.00,0.00,213.15,2.08,41.16,60026.0,209.481746,130.038346,130.038346,NaN,NaN,0.000000
2022-01-01 01:00:00-03:00,4.50,0.00,0.00,0.00,211.00,2.04,43.22,60026.0,191.989138,135.044884,135.044884,NaN,NaN,0.000000
2022-01-01 02:00:00-03:00,3.62,0.00,0.00,0.00,208.85,2.01,45.28,60020.0,172.342264,135.583713,135.583713,NaN,NaN,0.000000
2022-01-01 03:00:00-03:00,2.73,0.00,0.00,0.00,206.70,1.97,47.34,60009.0,154.114202,131.505209,131.505209,NaN,NaN,0.000000
2022-01-01 04:00:00-03:00,1.84,0.00,0.00,0.00,204.56,1.93,49.40,60026.0,139.612359,123.808202,123.808202,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00-03:00,9.82,140.50,351.81,42.75,223.88,2.28,30.87,59791.0,251.051728,73.855547,73.822162,180.0,45.000000,168.692114
2022-12-31 20:00:00-03:00,8.93,2.55,0.00,2.55,221.74,2.24,32.93,59885.0,246.514109,86.909960,86.778761,180.0,36.322609,0.000000
2022-12-31 21:00:00-03:00,8.04,0.00,0.00,0.00,219.59,2.20,34.99,59861.0,240.676531,99.508533,99.508533,NaN,NaN,0.000000


In [149]:
#  3. Validación y reindexado
expected_index = pd.date_range("2022-01-01", "2022-12-31 23:00", freq='H', tz=tz)
if len(met_df) < 8760:
    print(" Reindexando a 8760 horas...")
    met_df = met_df.reindex(expected_index)
    met_df[['ghi', 'dni', 'dhi']] = met_df[['ghi', 'dni', 'dhi']].fillna(0)
    met_df.fillna(method='ffill', inplace=True)
else:
    print(" met_df ya tiene 8760 registros")

 met_df ya tiene 8760 registros


C:\Users\Alvaro Henriquez\AppData\Local\Temp\ipykernel_9360\2439433765.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  expected_index = pd.date_range("2022-01-01", "2022-12-31 23:00", freq='H', tz=tz)


In [150]:
#  4. Calcular posiciones solares
location = Location(lat, lon, tz)
solpos = location.get_solarposition(met_df.index)

In [151]:
#  5. Función de estrategia de seguimiento solar
def get_tracking_angle(solpos_row, ghi, dhi, dni, strategy, albedo, gcr, limit_angle=60):
    azimuth = solpos_row['azimuth']
    zenith = solpos_row['zenith']
    if strategy == 'tracking_classic':
        axis_tilt = 0          # Inclinación del eje del seguidor (grados)
        axis_azimuth = 180     # Azimut del eje del seguidor (180° para norte-sur)
        max_angle = 60         # Límite del ángulo de inclinación del seguidor (grados)
        backtrack = True       # Activar backtracking

        # ⚠️ Corrección: pasar los valores individuales como Series de un solo elemento
        tracking_data = pvlib.tracking.singleaxis(
            apparent_zenith=pd.Series([zenith]),
            apparent_azimuth=pd.Series([azimuth]),
            axis_tilt=axis_tilt,
            axis_azimuth=axis_azimuth,
            max_angle=max_angle,
            backtrack=backtrack,
            gcr=gcr
        )

        # Devolver el único valor escalar del ángulo de seguimiento
        angle = tracking_data['tracker_theta'].values[0]

    elif strategy == 'plus5':
        axis_tilt = 0          # Inclinación del eje del seguidor (grados)
        axis_azimuth = 180     # Azimut del eje del seguidor (180° para norte-sur)
        max_angle = 60         # Límite del ángulo de inclinación del seguidor (grados)
        backtrack = True       # Activar backtracking

        # ⚠️ Corrección: pasar los valores individuales como Series de un solo elemento
        tracking_data = pvlib.tracking.singleaxis(
            apparent_zenith=pd.Series([zenith]),
            apparent_azimuth=pd.Series([azimuth]),
            axis_tilt=axis_tilt,
            axis_azimuth=axis_azimuth,
            max_angle=max_angle,
            backtrack=backtrack,
            gcr=gcr
        )

        # Devolver el único valor escalar del ángulo de seguimiento
        angle = tracking_data['tracker_theta'].values[0]+5
        angle = np.clip(angle, -60, 60)

    elif strategy == '80pct':
        axis_tilt = 0          # Inclinación del eje del seguidor (grados)
        axis_azimuth = 180     # Azimut del eje del seguidor (180° para norte-sur)
        max_angle = 60         # Límite del ángulo de inclinación del seguidor (grados)
        backtrack = True       # Activar backtracking

        # ⚠️ Corrección: pasar los valores individuales como Series de un solo elemento
        tracking_data = pvlib.tracking.singleaxis(
            apparent_zenith=pd.Series([zenith]),
            apparent_azimuth=pd.Series([azimuth]),
            axis_tilt=axis_tilt,
            axis_azimuth=axis_azimuth,
            max_angle=max_angle,
            backtrack=backtrack,
            gcr=gcr
        )

        # Devolver el único valor escalar del ángulo de seguimiento
        angle = tracking_data['tracker_theta'].values[0]*0.8
        angle = np.clip(angle, -60, 60)    
        
    elif strategy == 'maximize_irradiance':
        best_tilt, best_gain = 0, -np.inf
        for tilt in np.arange(-limit_angle, limit_angle+1, 1):
            tilt_rad = np.radians(tilt)
            gain = dni * np.cos(np.radians(zenith) - tilt_rad) \
                 + dhi * (1 + np.cos(tilt_rad)) / 2 \
                 + ghi * albedo * (1 - np.cos(tilt_rad)) / 2
            if gain > best_gain:
                best_gain, best_tilt = gain, tilt
        angle = best_tilt
    elif strategy == 'albedo_sensitive':
        angle = np.degrees(np.arctan2(np.sin(np.radians(azimuth - 180)), np.tan(np.radians(zenith))))
        if 60 < azimuth < 120:
            angle += albedo * 10
        elif 240 < azimuth < 300:
            angle -= albedo * 10
    else:
        raise ValueError(f"Estrategia no reconocida: {strategy}")
    return np.clip(np.round(angle / 5.0) * 5.0, -limit_angle, limit_angle)
    #return np.clip(np.round(angle / 60.0) * 60.0, -limit_angle, limit_angle)

In [152]:
#  6. Generar trackerdict usando estrategia seleccionada
trackerdict = {}
for i, time in enumerate(met_df.index):
    ghi = met_df.loc[time, 'ghi']
    dhi = met_df.loc[time, 'dhi']
    dni = met_df.loc[time, 'dni']
    solpos_row = solpos.loc[time]
 
    tilt = get_tracking_angle(solpos_row, ghi, dhi, dni, strategy, albedo, gcr=0.35)

    if np.isnan(tilt):
            print(f"⚠️  Skipping NaN tilt at {time}: zenith={solpos_row['zenith']}, azimuth={solpos_row['azimuth']}")
            continue

    if tilt not in trackerdict:
        trackerdict[tilt] = {
            'csvfile': f'EPWs/1axis_{tilt}.csv',
            'surf_azm': 0.0,
            'surf_tilt': tilt,
            'datetime': []
        }
    trackerdict[tilt]['datetime'].append(time)

⚠️  Skipping NaN tilt at 2022-01-01 00:00:00-03:00: zenith=130.05242148396587, azimuth=209.55712061268184
⚠️  Skipping NaN tilt at 2022-01-01 01:00:00-03:00: zenith=135.0761691238521, azimuth=192.0611468695307
⚠️  Skipping NaN tilt at 2022-01-01 02:00:00-03:00: zenith=135.6306744753138, azimuth=172.3955614737215
⚠️  Skipping NaN tilt at 2022-01-01 03:00:00-03:00: zenith=131.5617190602018, azimuth=154.14164653818182
⚠️  Skipping NaN tilt at 2022-01-01 04:00:00-03:00: zenith=123.86815334629202, azimuth=139.61904003518657
⚠️  Skipping NaN tilt at 2022-01-01 05:00:00-03:00: zenith=113.80623791296486, azimuth=128.81184871855805
⚠️  Skipping NaN tilt at 2022-01-01 06:00:00-03:00: zenith=102.29651733559201, azimuth=120.79003535008496
⚠️  Skipping NaN tilt at 2022-01-01 21:00:00-03:00: zenith=99.43232856377931, azimuth=240.85911064248307
⚠️  Skipping NaN tilt at 2022-01-01 22:00:00-03:00: zenith=111.20391012435455, azimuth=233.36797325732795
⚠️  Skipping NaN tilt at 2022-01-01 23:00:00-03:00: 

In [153]:
#  7. Crear archivos CSV por ángulo
for tilt, entry in trackerdict.items():
    df = pd.DataFrame(index=met_df.index, columns=['Beam', 'Diffuse'], data=0)
    df.loc[entry['datetime'], 'Beam'] = met_df.loc[entry['datetime'], 'dni']
    df.loc[entry['datetime'], 'Diffuse'] = met_df.loc[entry['datetime'], 'dhi']
    path = Path(testfolder) / entry['csvfile']
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(path, index=False)
    print(f" CSV para ángulo {tilt} generado: {entry['csvfile']}")

C:\Users\Alvaro Henriquez\AppData\Local\Temp\ipykernel_9360\867596449.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[   0.    738.2   931.48  968.73 1030.1  1109.74 1114.04 1102.14 1031.01
 1067.92  853.57 1084.81 1082.67 1065.68  727.87  841.    920.75    0.
  861.43    0.    903.06    0.      0.      0.      0.      0.      0.
    0.      0.      0.      0.      0.      0.      0.      0.      0.
    0.      0.      0.      0.      0.      0.      0.      0.      0.
    0.      0.      0.      0.      0.      0.      0.      0.      0.
    0.      0.      0.      0.      0.      0.      0.      0.      0.
    0.      0.      0.      0.      0.      0.      0.      0.      0.
    0.      0.      0.      0.      0.      0.      0.      0.      0.
    0.      0.      0.      0.      0.      0.      0.      0.      0.
    0.      0.      0.      0.      0.      0.      0.      0.      0.
    0.      

 CSV para ángulo -0.0 generado: EPWs/1axis_-0.0.csv
 CSV para ángulo -25.0 generado: EPWs/1axis_-25.0.csv
 CSV para ángulo -50.0 generado: EPWs/1axis_-50.0.csv
 CSV para ángulo -40.0 generado: EPWs/1axis_-40.0.csv
 CSV para ángulo -30.0 generado: EPWs/1axis_-30.0.csv
 CSV para ángulo -20.0 generado: EPWs/1axis_-20.0.csv
 CSV para ángulo -5.0 generado: EPWs/1axis_-5.0.csv
 CSV para ángulo 5.0 generado: EPWs/1axis_5.0.csv
 CSV para ángulo 15.0 generado: EPWs/1axis_15.0.csv


C:\Users\Alvaro Henriquez\AppData\Local\Temp\ipykernel_9360\867596449.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1184.09 1014.49  971.11 1172.14 1271.41 1176.86 1271.38 1163.23  818.1
  778.56  605.8   773.53 1222.02 1151.54 1221.87 1152.44  759.31  701.4
  568.45  729.32 1217.34 1123.38 1202.22 1147.77 1132.19 1124.82  749.03
 1042.29 1207.92 1162.1   704.08  643.71  642.96  924.81 1256.53 1036.03
 1162.86 1150.44 1142.4  1137.56 1164.29 1158.48 1159.62 1146.73 1101.66
 1146.22  853.93 1145.81 1125.21 1148.28 1189.74 1188.46 1166.44 1101.48
 1165.76 1157.99 1156.83 1171.01 1151.42  946.5   949.85 1031.08 1115.44
 1077.96  685.12  110.77  657.03  748.39  751.76  720.91  513.36  613.09
  321.86  743.37  382.1  1148.89 1105.9   904.62  547.41  880.77  355.78
  693.61 1020.93 1122.8  1098.61 1134.96 1137.29 1034.78  818.09  855.97
  866.39 1123.6   901.84 1143.84  991.26 1121.1  1138.38 1129.31 10

 CSV para ángulo 25.0 generado: EPWs/1axis_25.0.csv
 CSV para ángulo 35.0 generado: EPWs/1axis_35.0.csv
 CSV para ángulo 50.0 generado: EPWs/1axis_50.0.csv
 CSV para ángulo 30.0 generado: EPWs/1axis_30.0.csv
 CSV para ángulo 45.0 generado: EPWs/1axis_45.0.csv


C:\Users\Alvaro Henriquez\AppData\Local\Temp\ipykernel_9360\867596449.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[879.38 285.9  688.57 846.49 413.85 807.19 411.91 783.36 307.19 603.79
 325.98 687.9  280.15 567.91 325.19 633.72 339.61 201.66  97.7  312.73
 142.84 771.06 357.99 634.42 398.88 751.95 342.8  829.51 419.17 881.13
 412.16 186.29  71.55 269.37  85.39 417.65 204.46 793.77 309.8  385.74
 154.57 389.1  136.45 867.57 416.26 790.53 414.54 608.88 266.52 767.01
 308.   839.7  372.27 692.08 864.3  365.95 678.19 635.09 764.45 765.58
 862.06 829.51 627.94 648.56 668.86 875.02 708.46 840.22 462.47 603.75
 395.22 493.2  197.41 875.74 886.53 890.46 447.79 890.02 445.   896.44
 444.79 896.88 442.64 906.45 442.8  907.04 432.37 888.86 416.48 893.06
 596.99 876.98 875.66 878.44 549.61 628.42 872.83 865.   882.34 817.82
 873.14 870.37 875.55 870.29 851.97 847.48 851.32 607.86 858.76 857.52
 851.41 849.68

 CSV para ángulo -10.0 generado: EPWs/1axis_-10.0.csv
 CSV para ángulo -15.0 generado: EPWs/1axis_-15.0.csv
 CSV para ángulo -45.0 generado: EPWs/1axis_-45.0.csv
 CSV para ángulo -35.0 generado: EPWs/1axis_-35.0.csv
 CSV para ángulo 40.0 generado: EPWs/1axis_40.0.csv


C:\Users\Alvaro Henriquez\AppData\Local\Temp\ipykernel_9360\867596449.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1269.45 1269.23 1268.99 1268.75 1127.28  570.61  286.55  942.91 1267.35
 1152.55  823.24  876.22  974.61  192.63  845.92 1264.93 1246.39  871.47
 1263.74    0.      0.      0.      0.      0.      0.      0.      0.
    0.      0.      0.      0.      0.      0.      0.      0.      0.
    0.      0.      0.      0.      0.      0.      0.      0.      0.
    0.      0.      0.      0.      0.      0.   1072.49    0.   1122.96
    0.   1124.63    0.   1135.16    0.   1146.33    0.    923.25    0.
  952.41 1133.94 1141.65 1154.07 1143.99 1147.26 1147.27 1105.7   670.82
 1045.04 1147.52 1154.12 1161.7  1163.57 1158.46 1155.54 1139.9  1129.83
 1124.52 1032.2  1008.97 1114.05 1033.75 1106.02 1100.87 1026.27 1159.49
 1147.57 1155.97 1173.   1173.73 1168.66 1166.61 1169.96 1164.13  921.61


 CSV para ángulo 20.0 generado: EPWs/1axis_20.0.csv
 CSV para ángulo 10.0 generado: EPWs/1axis_10.0.csv


C:\Users\Alvaro Henriquez\AppData\Local\Temp\ipykernel_9360\867596449.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 105.6   101.67  107.78  107.08  121.46   92.3    93.1    65.41   89.38
   59.14   78.75   76.09   79.48   74.42   68.36   60.87   63.69   63.53
   65.12   67.25   65.96   68.04   70.64   73.19   71.91   74.4    68.9
   16.27   70.68   74.64   83.62   83.33   87.05   88.71   87.88   92.41
   92.52   84.97   80.86  903.55   90.41 1032.31   92.81  704.87   74.
 1021.01   95.15 1046.15  100.55 1047.28  104.49 1060.44  109.23 1068.73
  111.66  946.05   79.78  990.85  110.32 1062.18  117.28 1059.24  121.14
 1073.64  120.32 1073.73  121.55 1073.55  126.39 1070.96  127.89  875.73
   27.94  218.22    5.13 1064.83  132.41 1073.52  132.51 1060.75  135.63
 1085.63  140.55 1085.44  143.08 1077.64  143.68 1079.14 1066.6  1047.11
 1033.37  973.8   961.92  836.34  980.44 1028.51  990.52  931.94 107

In [154]:
#  8. Iniciar simulación bifacial
sceneDict = {'gcr': 0.35, 'hub_height': 2.3, 'nMods': 20, 'nRows': 8}
 
demo = bifacial_radiance.RadianceObj('BifacialSim', path=str(testfolder))
demo.setGround(albedo)
demo.metdata = bifacial_radiance.main.MetObj(
    met_df.rename(columns={'ghi': 'GHI', 'dhi': 'DHI', 'dni': 'DNI', 'temp_air': 'Tdry', 'Pressure': 'Pressure'}),
    metadata={
        'latitude': lat,
        'longitude': lon,
        'altitude': 2400,
        'TZ': -4,
        'Name': 'CustomData'
    }
)
 
demo.gencumskyArguments = '-r'
demo.genCumSky1axis(trackerdict=trackerdict)
trackerdict = demo.makeScene1axis(trackerdict=trackerdict, module='test-module', sceneDict=sceneDict)
trackerdict = demo.makeOct1axis(trackerdict=trackerdict)
demo.cumulativesky = True

path = C:\Users\Alvaro Henriquez\OneDrive - fraunhofer.cl\Escritorio\FCR\Bifacial Radiance Simulations\BifacialSimulationsSWC2025\Post_SWC\Pruebas_SAB
Loading albedo, 1 value(s), 0.700 avg
1 nonzero albedo values.
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
message: There were 1 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-50.0.rad
message: There were 1 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-45.0.rad
message: There were 1 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-40.0.rad
message: There were 1 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-35.0.rad
message: There were 1 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-30.0.rad
message: There were 1 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Create

In [ ]:
#  9. Análisis para un módulo específico
modWanted = 9
rowWanted = 2
customname = 'Row_2_Module_09'
trackerdict = demo.analysis1axis(trackerdict, modWanted=modWanted, rowWanted=rowWanted, customname=customname)

Linescan in process: 1axis_-50.0Row_2_Module_09_Front
Linescan in process: 1axis_-50.0Row_2_Module_09_Back
Saved: results\irr_1axis_-50.0Row_2_Module_09.csv
Index: -50.0. Wm2Front: 0.0. Wm2Back: 0.0
Linescan in process: 1axis_-45.0Row_2_Module_09_Front
Linescan in process: 1axis_-45.0Row_2_Module_09_Back
Saved: results\irr_1axis_-45.0Row_2_Module_09.csv
Index: -45.0. Wm2Front: 0.0. Wm2Back: 0.0
Linescan in process: 1axis_-40.0Row_2_Module_09_Front
Linescan in process: 1axis_-40.0Row_2_Module_09_Back
Saved: results\irr_1axis_-40.0Row_2_Module_09.csv
Index: -40.0. Wm2Front: 0.0. Wm2Back: 0.0
Linescan in process: 1axis_-35.0Row_2_Module_09_Front
Linescan in process: 1axis_-35.0Row_2_Module_09_Back
Saved: results\irr_1axis_-35.0Row_2_Module_09.csv
Index: -35.0. Wm2Front: 0.0. Wm2Back: 0.0
Linescan in process: 1axis_-30.0Row_2_Module_09_Front
Linescan in process: 1axis_-30.0Row_2_Module_09_Back
Saved: results\irr_1axis_-30.0Row_2_Module_09.csv
Index: -30.0. Wm2Front: 0.0. Wm2Back: 0.0
Lines

In [ ]:
#  10. Leer resultados
results = load.read1Result(f'cumulative_results_{customname}.csv')

In [ ]:
results_clean = load.cleanResult(results)
results_clean

In [ ]:
# Inicializar una lista para almacenar los resultados
results = []

# Bucle para probar diferentes valores de bifacialityfactor
for bifacialityfactor in np.arange(0.0, 1.05, 0.05):
    # Cálculos de las variables
    annual_front_side = np.nanmean(results_clean.Wm2Front)
    annual_back_side = np.nanmean(results_clean.Wm2Back)* bifacialityfactor
    annual_total = annual_front_side + annual_back_side* bifacialityfactor
    annual_bifacial_ratio = (annual_back_side * bifacialityfactor) / annual_front_side

    # Almacenar los resultados en una lista
    results.append([bifacialityfactor, annual_front_side, annual_back_side, annual_total, annual_bifacial_ratio,albedo, strategy])

# Convertir los resultados en un DataFrame
results_df = pd.DataFrame(results, columns=['Bifaciality Factor', 'Annual Front Side (W/m2)', 'Annual Back Side (W/m2)', 'Annual Total (W/m2)', 'Annual Bifacial Ratio', 'Albedo', 'Tracking Strategy'])

# Mostrar la tabla de resultados
print(results_df)

# Guardar los resultados en un archivo CSV
results_df.to_csv(f'bifaciality_factor_analysis_{strategy}.csv', index=False)

In [ ]:
def extract_bifacial_data(directory, output_filename='resultados.csv'):
    data_list = []
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            file_path = os.path.join(directory, file)
            try:
                df = pd.read_csv(file_path)
                # Extraer las columnas específicas
                extracted = df[['Bifaciality Factor', 'Albedo', 'Tracking Strategy', 'Annual Bifacial Ratio','Annual Front Side (W/m2)','Annual Back Side (W/m2)']]
                extracted['Filename'] = file  # Añadir el nombre del archivo como columna
                data_list.append(extracted)
            except Exception as e:
                print(f"Error al procesar {file}: {e}")
    # Concatenar todos los resultados en un solo DataFrame llamado 'resultados'
    if data_list:
        resultados = pd.concat(data_list, ignore_index=True)
        # Guardar el DataFrame en un archivo CSV
        resultados.to_csv(output_filename, index=False)
        print(f"Archivo CSV guardado como: {output_filename}")
        return resultados
    else:
        print("No se encontraron archivos CSV o no se pudieron procesar.")
        return None

In [ ]:
#extract_bifacial_data("C:\\Users\\Alvaro Henriquez")
extract_bifacial_data("C:\\Users\Alvaro Henriquez\\OneDrive - fraunhofer.cl\\Escritorio\\FCR\\Bifacial Radiance Simulations\\BifacialSimulationsSWC2025\\Post_SWC\\Pruebas_SAB")

In [ ]:
def load_results_csv(filename='resultados.csv'):
    try:
        df_resultados = pd.read_csv(filename)
        print(f"Archivo {filename} cargado exitosamente.")
        print(df_resultados.head())  # Mostrar las primeras filas para verificar
        return df_resultados
    except Exception as e:
        print(f"Error al cargar el archivo {filename}: {e}")
        return None

In [ ]:
df1=load_results_csv(filename='resultados.csv')

In [ ]:
df1

In [ ]:
# Se elige el factor de bifacialidad (que depende del tipo de módulo) para hacer el gráfico
# Longi PERC+ 0.60 - 0.80
# Canadian Solar nPERT 0.80 - 0.95
# Jinko Solar HJT 0.90 - 0.95
# SANYO TopCon 0.85 - 0.90
# LG IBC 0.60 -0.75

#bifaciality_factor=0.6000000000000001
bifaciality_factor=0.8
#bifaciality_factor=0.9
#bifaciality_factor=0.8500000000000001

In [ ]:
def plot_bifacial_ratio_estrategias_puntos_total_tendencia(resultados, bifaciality_factor):
    # Filtrar el DataFrame por el valor dado de Bifaciality Factor
    filtered_df = resultados[resultados['Bifaciality Factor'] == bifaciality_factor]

    if filtered_df.empty:
        print(f"No se encontraron datos para el Bifaciality Factor: {bifaciality_factor}")
        return

    # Crear el gráfico de puntos con etiquetas por estrategia de tracking
    plt.figure(figsize=(10, 6))
    unique_strategies = filtered_df['Tracking Strategy'].unique()
    colors = plt.cm.tab10.colors  # Paleta de colores

    for i, strategy in enumerate(unique_strategies):
        strategy_df = filtered_df[filtered_df['Tracking Strategy'] == strategy]
        x = strategy_df['Albedo']
        y = strategy_df['Annual Front Side (W/m2)'] + strategy_df['Annual Back Side (W/m2)']*bifaciality_factor

        # Color por estrategia
        color = colors[i % len(colors)]

        # Puntos
        plt.plot(x, y, marker='o', linestyle='', color=color, label=strategy)

        if len(x) > 1:
            # Ajuste lineal
            coef = np.polyfit(x, y, 1)
            poly = np.poly1d(coef)
            y_fit = poly(x)

            # R²
            r_squared = 1 - np.sum((y - y_fit) ** 2) / np.sum((y - np.mean(y)) ** 2)

            # Línea de tendencia
            x_sorted = np.sort(x)
            plt.plot(x_sorted, poly(x_sorted), linestyle='--', linewidth=1.5, color=color)

            # Mostrar ecuación y R² en el gráfico
            x_text = x_sorted.iloc[-1] if hasattr(x_sorted, "iloc") else x_sorted[-1]
            y_text = poly(x_text)
            plt.text(x_text, y_text,
                     f"${coef[0]:.2f}x + {coef[1]:.2f}$\n$R^2 = {r_squared:.3f}$",
                     fontsize=9, color=color, ha='right', va='bottom')

    plt.title(f'Annual Total Irradiance on Module vs Albedo (Bifaciality Factor: {bifaciality_factor})')
    plt.xlabel('Albedo')
    plt.ylabel('Annual Total Irradiance on Module')
    plt.legend(loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_bifacial_ratio_estrategias_puntos_total_tendencia(df1, bifaciality_factor)

In [ ]:
def plot_bifacial_ratio_estrategias_puntos_total(resultados, bifaciality_factor):
    # Filtrar el DataFrame por el valor dado de Bifaciality Factor
    filtered_df = resultados[resultados['Bifaciality Factor'] == bifaciality_factor]

    if filtered_df.empty:
        print(f"No se encontraron datos para el Bifaciality Factor: {bifaciality_factor}")
        return

    # Crear el gráfico de área apilada separado por estrategia de tracking
    plt.figure(figsize=(10, 6))
    unique_strategies = filtered_df['Tracking Strategy'].unique()

    for strategy in unique_strategies:
        strategy_df = filtered_df[filtered_df['Tracking Strategy'] == strategy]
        plt.plot(strategy_df['Albedo'], strategy_df['Annual Front Side (W/m2)'] + bifaciality_factor * strategy_df['Annual Back Side (W/m2)'], marker='o', linestyle='')

        # Añadir etiquetas en cada punto
        for x, y in zip(strategy_df['Albedo'], strategy_df['Annual Front Side (W/m2)'] + bifaciality_factor * strategy_df['Annual Back Side (W/m2)']):
            plt.text(x, y, f'{y:.2f}', fontsize=8, ha='right', va='bottom')

    plt.title(f'Bifacial Ratio vs Albedo (Bifaciality Factor: {bifaciality_factor})')
    plt.xlabel('Albedo')
    plt.ylabel('Annual Bifacial Ratio')
    plt.legend(loc='upper left')
    plt.grid(True)
    plt.show()

In [ ]:
#bifaciality_factor=0.6000000000000001
#bifaciality_factor=0.8
#bifaciality_factor=0.9
#bifaciality_factor=0.8500000000000001
bifaciality_factor=0.0
plot_bifacial_ratio_estrategias_puntos_total(df1, bifaciality_factor)